In [ ]:
import sys
path1 = '/Users/timkrausz/Documents/GitHub/DA_maze_paper/DA/Modules/'
path2 = '/Users/timkrausz/Documents/GitHub/DA_maze_paper/Behavior/Modules/'
sys.path += [path1,path2]

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-poster")
from multi_rat_da import *
from hexLevelAnalyses import get_sigRats_fromMeanList
from photometryQuantifications import *
from scipy.stats import wilcoxon
%matplotlib qt

### load trial-level dataframe

In [2]:
loadpath = "" #location of dataset
df = pd.read_csv(loadpath+"triframe.csv")
photrats = PhotRats(None)
photrats.triframe = df.copy()
photrats.triframe.reset_index(inplace=True)
del df
photrats.directory_prefix = loadpath

## create new dataframes with relevant columns for regression, and plot results

In [7]:
get_portHist(photrats)

get_back2samePortTrials(photrats)

add_rewardLagColumns2triframe(photrats)

add_rewardLagColumns2triframeByPort(photrats,lags=5)

add_samePathLagColumns2triframeByPort(photrats,lags=1)

add_leftLastColumn(photrats)

### identify whether rat previously took the same or alternative path to the left available port, and if it was rewarded or not

In [12]:
for l in range(1,6):
    photrats.triframe.loc[:,"rt-"+str(l)+"_left"] = photrats.get_left_val(photrats.triframe,factor="rt-"+str(l))
photrats.triframe.loc[:,"samePath_t-1_left"] = photrats.get_left_val(photrats.triframe,factor="samePath_t-1")

### save as csv for regression analysis in R

In [29]:
photrats.triframe.to_csv(photrats.directory_prefix+"triframe4sameValtLeftChoiceReg.csv")

### create a dictionary of reward vs omission choice probabilities when rat previously took same vs alternative path

In [15]:
ratSameValtChoiceProbs,ratNs = create_sameValtChoiceDict(photrats)

### test if same and alt are sig dif from each other

In [24]:
sameVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmSame"]) for rat in ratSameValtChoiceProbs.keys()]
altVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmAlt"]) for rat in ratSameValtChoiceProbs.keys()]
wilcoxon(sameVals,altVals)

WilcoxonResult(statistic=16.0, pvalue=0.275390625)

### visualize result

In [ ]:
fig = plotChoosePortSameVAlt(photrats,ratSameValtChoiceProbs)
plt.ylim(-.3,.3)
plt.yticks(np.arange(-3,4)/10)

# visualize individual rat sensitivities to costs and benefits

In [11]:
get_log_pchoos_v_costNben(photrats)
add_scaledVarsToRegDf(photrats)
photrats.regdf = photrats.regdf.loc[photrats.regdf.tri>25,]

In [39]:
plot_choiceSigmoidsByRelativeCostsAndBen(photrats)

### plot individual regression weights for costs and benefits for each rat

In [13]:
plot_choiceRegWeightsByRat(photrats)

Optimization terminated successfully.
         Current function value: 0.534518
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.547970
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.560840
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591582
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.575395
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.500732
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.536611
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.560975
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.630448
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620402
  

- adding significance test to individual rat regression weight calculation

In [36]:
regWeightsByRat.values[:,3:]<0.05

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

### get and save dataframe for mixed-effects p(choose L) regression in R

In [34]:
get_log_pchoos_v_costNben(photrats)
add_scaledVarsToRegDf(photrats)
photrats.regdf.to_csv(photrats.directory_prefix+"costVbenDf4reg.csv")

### load regression results from R

In [46]:
regCoefs = pd.read_csv(photrats.directory_prefix+"chooseLregCoefsByRat.csv")
regSum = pd.read_csv(photrats.directory_prefix+"chooseLregSummary.csv")

### plot regression results from R

In [50]:
regSum

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),2.214101,0.309209,7.160538,8.036102e-13
1,pRwdScaled,1.613729,0.158150,10.203759,1.907441e-24
2,ldifScaled,-6.803168,0.548637,-12.400121,2.609282e-35


In [53]:
fig = plt.figure(figsize=(3.5,4))
xvals = ["p(reward)","Distance"]
sns.barplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='darkgrey',alpha=.5)
sns.stripplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='k',size=8,marker='X',alpha=.9)
plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals,fontweight="bold")
plotRegSigLevel(regSum,0,2.0,1)
plotRegSigLevel(regSum,1,2.0,2)
plt.ylim(-10,2.2)
plt.ylabel("Path choice ß value",fontsize='xx-large',fontweight='bold')
plt.tight_layout()

# plot change in port/path choice around block change

In [40]:
high2lowProb,high2lowDist,low2highProb,low2highDist = get_sessionProbAndDistChangeInds(photrats)

In [ ]:
lowsProb,highsProb,lowcountProb,highcountProb = calc_pChoosePortAtProbBlkTrans(photrats,high2lowProb,low2highProb)
fig = plot_portChoiceAtProbBlkTrans(photrats,lowsProb,highsProb,smoothWin=5,blineStart=5,legend_on=False)
plt.ylim(-.18,.18)

In [ ]:
lowsBar,highsBar,lowcountBar,highcountBar = calc_pChoosePathAtBarBlkTrans(photrats,high2lowDist,low2highDist)
fig = plot_portChoiceAtBarBlkTrans(photrats,lowsBar,highsBar,smoothWin=5,blineStart=5,legend_on=False)